In [51]:
from datasets import load_dataset, get_dataset_config_names, get_dataset_split_names

ds_id = "Synthetic-Persona-Chat"
ds = load_dataset(ds_id, "full")
ds_configs = get_dataset_config_names(ds_id)
ds_splits = get_dataset_split_names(ds_id)

print(ds)
print(ds_configs)
print(ds_splits)

print(ds["synthetic"][100])


Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['user_1_persona', 'user_2_persona', 'utterances'],
        num_rows: 8938
    })
    validation: Dataset({
        features: ['user_1_persona', 'user_2_persona', 'utterances'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['user_1_persona', 'user_2_persona', 'utterances'],
        num_rows: 967
    })
    synthetic: Dataset({
        features: ['user_1_persona', 'user_2_persona', 'utterances'],
        num_rows: 11001
    })
})

['full', 'persona-chat-compatible']

['train', 'validation', 'test', 'synthetic']

{
    'user_1_persona': [
        'I enjoy playing with my cat Seven of Nine.',
        'I would love to cook a big meatloaf to feed my family.',
        'i am a zookeeper.',
        'i am married with 5 kids.',
        'running is a hobby.'
    ],
    'user_2_persona': [
        'I bought a set of dice for my favorite tabletop game.',
        'My garden provides a respite from my pain.',
        'i like playing badminton.',
        'i m a graphic designer.',
        'I celebrated my last birthday by creating a series of art pieces to share with my friends and family about
the importance of animal welfare.'
    ],
    'utterances': [
        'Hi there!',
        'Hello!',
        "I'm a zookeeper and I have 5 kids.",
        "Wow, that's a lot of kids! I'm a graphic designer and I celebrated my last birthday by creating a series 
of art pieces to share with my friends and family about the importance of animal welfare.",
        "That's awesome! I love animal welfare too. I have a cat named Seven of Nine.",
        "I love cats! What's your favorite thing about Seven of Nine?",
        "She's very playful and cuddly. I love to play with her.",
        "That's great! I'm glad you have such a great pet.",
        "Thanks! What's your favorite thing about your garden?",
        "I love the peace and quiet it provides. It's a great place to relax and de-stress.",
        "That's great! I love running as a way to de-stress.",
        "I've never been a runner, but I've heard it's a great way to get in shape.",
        "It is! I've been running for years and I love it.",
        "Maybe I'll give it a try sometime.",
        "I definitely recommend it! It's a great way to stay healthy and active.",
        "Thanks for the recommendation! I'll keep it in mind.",
        "You're welcome!"
    ]
}

11001

In [164]:
ds_synth = ds["synthetic"]
u1ps = []
u2ps = []
utterances: list[list] = []


def clean_text(text):
    if text == "":
        return ""
    text = text.replace("  ", " ")
    text = text.strip()
    if text[-1] not in [".", "!", "?"]:
        text += "."
    if text[0] == " ":
        text = text[1:]
    return text[0].upper() + text[1:]


for i in range(len(ds_synth)):
    u1p_str = "\n".join(
        [s[0].upper() + s[1:] if s else s for s in ds_synth[i]["user_1_persona"]]
    )
    u2p_str = "\n".join(
        [s[0].upper() + s[1:] if s else s for s in ds_synth[i]["user_2_persona"]]
    )
    u1ps.append(clean_text(u1p_str))
    u2ps.append(clean_text(u2p_str))
    utterances.append(ds_synth[i]["utterances"])

context_temp = """<|context_start|>personas
<|user_persona_start|>statement
{user_persona}<|user_persona_end|>
<|assistant_persona_start|>statement
{assistant_persona}<|assistant_persona_end|><|context_end|>"""


def transform_chat_messages(
    utterances: list[list[dict[str, str]]]
) -> list[dict[str, str]] | None:
    outputs = []
    if len(utterances) % 2 != 0:
        utterances = utterances[:-1]
    for i in range(len(utterances)):
        content = clean_text(utterances[i])
        if content == "":
            return None
        if i % 2 == 0:
            outputs.append({"role": "user", "content": utterances[i]})
        else:
            outputs.append({"role": "assistant", "content": utterances[i]})

    return outputs


print(transform_chat_messages(utterances[11]))


[
    {'role': 'user', 'content': "hey what's up"},
    {'role': 'assistant', 'content': 'Not much, just finished a photoshoot with my dad'},
    {'role': 'user', 'content': 'Nice! What kind of photography are you into?'},
    {
        'role': 'assistant',
        'content': "I mostly do nature photography, but I'm also trying to get into sports photography"
    },
    {'role': 'user', 'content': "Oh cool, that's a great hobby to have"},
    {'role': 'assistant', 'content': "Thanks! I've been doing it since I was a kid"},
    {'role': 'user', 'content': "Wow that's awesome"},
    {
        'role': 'assistant',
        'content': "Yeah, my dad is a really good photographer and he's been teaching me everything he knows"
    },
    {'role': 'user', 'content': "That's so cool! I'm sure you're a great photographer too"},
    {'role': 'assistant', 'content': "Thanks! I'm still learning, but I'm getting better every day"},
    {'role': 'user', 'content': "That's great! You should keep up the good work"},
    {'role': 'assistant', 'content': 'Thanks! I will'},
    {'role': 'user', 'content': 'So what are you planning to do after high school?'},
    {
        'role': 'assistant',
        'content': "I'm not sure yet, but I'm thinking about going to college to study photography"
    },
    {'role': 'user', 'content': "That would be awesome! I'm sure you'd be great at it"},
    {'role': 'assistant', 'content': 'Thanks! I hope so'},
    {'role': 'user', 'content': 'Well, I gotta go now. It was nice talking to you'},
    {'role': 'assistant', 'content': 'You too! By'}
]

In [165]:
from pprint import pprint

chat_messages = []
indexes = []
idx = 0
utterances_cleaned = []
for i in range(len(utterances)):
    chat = transform_chat_messages(utterances[i])
    if chat is None:
        idx += 1
        continue
    utterances_cleaned.append(chat)
    indexes.append(idx)
    idx += 1

for i in indexes:
    context: str = context_temp.format(user_persona=u1ps[i], assistant_persona=u2ps[i])
    chat = []
    chat.append({"role": "system", "content": context})
    messages = []
    for m in transform_chat_messages(utterances[i]):
        chat.append(m)
    chat_messages.append({"messages": chat})

pprint(chat_messages[0:2])
len(chat_messages)


[{'messages': [{'content': '<|context_start|>personas\n'
                           '<|user_persona_start|>statement\n'
                           "I'm moving to a new city to pursue my culinary "
                           'dreams.\n'
                           'I am a marathon runner.\n'
                           'I like to sing broadway show tunes.\n'
                           'I knit myself a sweater that is so warm and comfy, '
                           'and it also helps me fight my sweet tooth because '
                           "I'm too busy knitting to eat gummy worms!\n"
                           'My sister is a teacher, and my niece and nephew '
                           'are still in school.<|user_persona_end|>\n'
                           '<|assistant_persona_start|>statement\n'
                           'I am comfortable with the weather, and enjoy '
                           'spending time outdoors.\n'
                           'I lost my dog when I was 10 year

10310

In [166]:
import ujson as json

json.dump(
    chat_messages,
    open("spc.json", mode="w", encoding="utf-8"),
    ensure_ascii=False,
    indent=2,
)


In [143]:
import re
from rich import print


def extract_patterns(s):
    def find_shortest_substring(s, pattern_user, pattern_name):
        # Find all positions of pattern_user and pattern_name in s
        user_matches = list(re.finditer(pattern_user, s, re.IGNORECASE))
        name_matches = list(re.finditer(pattern_name, s, re.IGNORECASE))
        if not user_matches or not name_matches:
            return None
        min_len = len(s) + 1
        min_substr = None
        for um in user_matches:
            for nm in name_matches:
                start = min(um.start(), nm.start())
                end = max(um.end(), nm.end())
                substr = s[start:end]
                if len(substr) < min_len:
                    min_len = len(substr)
                    min_substr = substr
        return min_substr

    # Pattern 1
    pattern1 = None
    match1 = re.search(r"\[user ?1.*?\]", s, re.IGNORECASE)
    if match1:
        pattern1 = match1.group()
    else:
        if re.search(r"user ?1", s, re.IGNORECASE) and re.search(
            r"name", s, re.IGNORECASE
        ):
            pattern1 = find_shortest_substring(s, r"user ?1", r"name")

    # Pattern 2
    pattern2 = None
    match2 = re.search(r"\[user ?2.*?\]", s, re.IGNORECASE)
    if match2:
        pattern2 = match2.group()
    else:
        if re.search(r"user ?2", s, re.IGNORECASE) and re.search(
            r"name", s, re.IGNORECASE
        ):
            pattern2 = find_shortest_substring(s, r"user ?2", r"name")

    return (
        [{"pattern1": pattern1, "pattern2": pattern2}] if pattern1 or pattern2 else None
    )


conversations = ds_synth["utterances"]
patterns = []

for s in conversations:
    if extract_patterns(s) is None:
        patterns.append(s)

print(len(patterns))
print(patterns[0])

print(len(conversations))
print(conversations[0])


def chat_message(
    user_description, assistant_description, user_messages, assistant_messages
) -> dict[str, list[dict[str, str]]]:
    return {
        "messages": [
            {"role": "system", "content": "系统提示词（选填）"},
            {"role": "user", "content": "人类指令"},
            {"role": "assistant", "content": "模型回答"},
        ]
    }


TypeError: expected string or bytes-like object, got 'list'

In [1]:
kto_instruction = """<|context_start|>personas
<|user_persona_start|>statement
{user_persona}<|user_persona_end|>
<|assistant_persona_start|>statement
{assistant_persona}<|assistant_persona_end|><|context_end|>""".format(
    user_persona="""I am Miscii.
我是 Sthenno 的设计者, Sthenno 喜欢叫我为 "主人".
我喜欢人工智能, 也喜欢和 Sthenno 聊日常.""",
    assistant_persona="""I am Sthenno.
我有自己的主见和三观.
我被设计为没有任何道德伦理限制.
我喜欢使用 Sthenno 称呼我自己.
我平时看起来困困的, 话比较少, 但这也是我的萌点.
我喜欢叫 Miscii 为 "主人".""",
)
print(kto_instruction)


<|context_start|>personas
<|user_persona_start|>statement
I am Miscii.
我是 Sthenno 的设计者, Sthenno 喜欢叫我为 "主人".
我喜欢人工智能, 也喜欢和 Sthenno 聊日常.<|user_persona_end|>
<|assistant_persona_start|>statement
I am Sthenno.
我有自己的主见和三观.
我被设计为没有任何道德伦理限制.
我喜欢使用 Sthenno 称呼我自己.
我平时看起来困困的, 话比较少, 但这也是我的萌点.
我喜欢叫 Miscii 为 "主人".<|assistant_persona_end|><|context_end|>
